# Edges Detection and Contoures

## Introduction

One of the most important features of a frame is Objects Edges and Contoures; by tracing the contours around objects, image processing algorithms can detect and recognize objects' shapes and track them.

Canny algorithm find edges by detecting sudden changes in pixel intensity (i.e. sharp transitions in colors or grayscale).

In this experiment, we'll detect the edges (i.e. using Canny Algorithm) and find the contoures (i.e. external contour) from them, then we can draw only the contours with length in a specific range.

We convert an input image to its edges using Canny edge detection algorithm with some additional preprocessing steps:

- Determine Lower and Upper Thresholds: by caclulating median intensity for the image; add and substract some value from it depending on a multiplier value to calculate lower and upper thresholds (e.g. if `multiplier=0.3`, we put `70%` of the median intensity value for lower threshold and `130%` of the median intensity value for the upper threshold).

- Apply Blur using a specified blur kernel size (3\*3): blur helps to smooth the image and reduce noise, which can improve the accuracy of edge detection.

Once we get the edges we find and draw the External Contours. then we tried to apply some Morphological operations to reduse the Noise.

## Implementation

In [ ]:
from expt_utils import *

### Edges Detection (with Canny Algorithm)

In [ ]:
def img_to_canny_edges(img, blur_kernel=None, multiplier=0.3):
    """
    Detect the edges of the given `img` using Canny algorithm,
    the upper and lower values are calculated depending on img
    median value plus and minus some `multiplier` of it.

    You can bluring the image before applying Canny by parsing
    a `blur_kernel` to it (e.g. blur_kernel=(5, 5)).

    You can change `blur_kernel` size or `multiplier` value to
    control how much detailes are detected.
    """
    if blur_kernel:
        img = cv.blur(img, blur_kernel)
    img_med = np.median(img)
    img_med_lower = int(max(0, (1 - multiplier) * img_med))
    img_med_upper = int(min(255, (1 + multiplier) * img_med))
    img_edges = cv.Canny(img, img_med_lower, img_med_upper)
    return img_edges

In [ ]:
img = plt.imread(f'{DS_DIR}/frames/train/00012/00012_1560.jpg')
img_edges = img_to_canny_edges(img, blur_kernel=(3, 3), multiplier=.50)

plt.figure(figsize=(10, 3.5), tight_layout=True)

plt.subplot(1, 2, 1), plt.axis('off')
plt.title('Image')
plt.imshow(img)

plt.subplot(1, 2, 2), plt.axis('off')
plt.title('Canny Edges (blur kernel: (3, 3), multiplier: .50)')
plt.imshow(img_edges, cmap='gray')

plt.savefig(f'{OUT_DIR}/01-01-canny_edges')

In [ ]:
def _idx_to_kernel(idx):
    return None if idx == 0 else (idx * 2 + 1, idx * 2 + 1)


def _render(blur_kernel, multiplier):
    img_edges = img_to_canny_edges(img, blur_kernel, multiplier)
    cv.imshow('Canny Edges', img_edges)


def _update_blur_kernel(idx):
    _render(
        blur_kernel=_idx_to_kernel(idx),
        multiplier=cv.getTrackbarPos('Multiplier', 'Canny Edges'))


def _update_multiplier(idx):
    _render(
        blur_kernel=_idx_to_kernel(cv.getTrackbarPos('Blur Kernel', 'Canny Edges')),
        multiplier=(0.01 * idx))


_render(blur_kernel=None, multiplier=0.3)
cv.createTrackbar('Blur Kernel', 'Canny Edges', 0, 5, _update_blur_kernel)
cv.createTrackbar('Multiplier', 'Canny Edges', 30, 100, _update_multiplier)

destroy_when_esc()

### Find Contours from Edges

> To add some control, we can select only contours with length in a specific range (i.e. from `rng_down` to `rng_up`).

In [ ]:
def edges_to_contours(edges, rng_down=0, rng_up=np.inf, color=(255, 0, 0), thickness=1):
    """
    Find and draw contours from a given `edges`, the contours
    are drawen on black image with the given `color` and
    `thickness`.
    
    You can change `rng_down` and `rng_up` to determain the
    range that contours' lengths are belong to.
    """
    all_contours = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)[0]
    selected_contours = [c for c in all_contours if rng_down < len(c) < rng_up]
    img_contours = np.zeros((*edges.shape[:2], 3)).astype('uint8')
    cv.drawContours(img_contours, selected_contours, -1, color, thickness)
    return img_contours

In [ ]:
img_edges = img_to_canny_edges(img, blur_kernel=(5, 5), multiplier=.50)
img_contours = edges_to_contours(img_edges, rng_down=20, rng_up=250)


plt.figure(figsize=(10, 3.5), tight_layout=True)

plt.subplot(1, 2, 1), plt.axis('off')
plt.title('Canny Edges (blur kernel: (5, 5), multiplier: .50)')
plt.imshow(img_edges, cmap='gray')

plt.subplot(1, 2, 2), plt.axis('off')
plt.title('Contours (length range: [20, 250])')
plt.imshow(img_contours, cmap='gray')

plt.savefig(f'{OUT_DIR}/01-02-edges_contours')

In [ ]:

def _render(blur_kernel, multiplier, rng_down, rng_up):
    img_edges = img_to_canny_edges(img, blur_kernel, multiplier)
    img_edges_contours = edges_to_contours(img_edges, rng_down, rng_up)
    img_edges_contours_bgr = cv.cvtColor(img_edges_contours, cv.COLOR_BGR2RGB)
    cv.imshow('Contours', img_edges_contours_bgr)


def _update_blur_kernel(idx):
    _render(
        blur_kernel=_idx_to_kernel(idx),
        multiplier=cv.getTrackbarPos('Multiplier', 'Contours'),
        rng_down=cv.getTrackbarPos('Range Down', 'Contours'),
        rng_up=cv.getTrackbarPos('Range Up', 'Contours'))


def _update_multiplier(idx):
    _render(
        blur_kernel=_idx_to_kernel(
            cv.getTrackbarPos('Blur Kernel', 'Contours')),
        multiplier=(0.01 * idx),
        rng_down=cv.getTrackbarPos('Range Down', 'Contours'),
        rng_up=cv.getTrackbarPos('Range Up', 'Contours'))


def _update_rng_down(idx):
    _render(
        blur_kernel=_idx_to_kernel(
            cv.getTrackbarPos('Blur Kernel', 'Contours')),
        multiplier=0.01 * cv.getTrackbarPos('Multiplier', 'Contours'),
        rng_down=int(cv.getTrackbarPos('Range Down', 'Contours')),
        rng_up=idx)


def _update_rng_up(idx):
    _render(
        blur_kernel=_idx_to_kernel(
            cv.getTrackbarPos('Blur Kernel', 'Contours')),
        multiplier=0.01 * cv.getTrackbarPos('Multiplier', 'Contours'),
        rng_down=idx,
        rng_up=int(cv.getTrackbarPos('Range Up', 'Contours')))


_render(0, .30, 0, 9999)
cv.createTrackbar('Blur Kernel', 'Contours', 0, 5, _update_blur_kernel)
cv.createTrackbar('Multiplier', 'Contours', 30, 100, _update_multiplier)
cv.createTrackbar('Range Down', 'Contours', 0, 9999, _update_rng_down)
cv.createTrackbar('Range Up', 'Contours', 9999, 9999, _update_rng_up)


destroy_when_esc()

### Enhancing Contours by Morphological Operations

For more enhancements we apply CLOSE morph operation on contour image.

In [ ]:
img_edges = img_to_canny_edges(img, blur_kernel=(5, 5), multiplier=.50)
img_contours = edges_to_contours(img_edges, rng_down=20, rng_up=250)
img_edges_morph = cv.morphologyEx(img_edges, cv.MORPH_CLOSE, np.ones((5, 5), np.uint8))
img_contours_morph = edges_to_contours(img_edges_morph)


plt.figure(figsize=(10, 7), tight_layout=True)

plt.subplot(2, 2, 1), plt.axis('off'), plt.title('Edges')
plt.imshow(img_edges, cmap='gray')

plt.subplot(2, 2, 2), plt.axis('off'), plt.title('Contours')
plt.imshow(img_contours)

plt.subplot(2, 2, 3), plt.axis('off'), plt.title('Edges with Morph')
plt.imshow(img_edges_morph, cmap='gray')

plt.subplot(2, 2, 4), plt.axis('off'), plt.title('Contours with Morph')
plt.imshow(img_contours_morph)

plt.savefig(f'{OUT_DIR}/01-03-morph_edges_contours')

By applying Morphological operation we can see clearly that it refine the contours of objects by filling gaps, closing small breaks, and smoothing jagged edges. This enhances the completeness and continuity of contours, making them more suitable for precise object recognition.